### RAG with Weaviate Vector store

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
embeddings_google = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings_google

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x105ae1d10>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [4]:
len(embeddings_google.embed_query("Hello AI"))

768

In [5]:
# Google LLM
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [6]:
# Prompt
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt.messages

/Users/I572648/Library/CloudStorage/OneDrive-SAPSE/Desktop/Git/GenAI/.venv/lib/python3.11/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [7]:
# Weaviare Vector Store
import weaviate

local_client = weaviate.connect_to_local()
local_client

In [9]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders.text import TextLoader

loader = TextLoader("../data/sample.txt")
text_document = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=20)
texts = text_splitter.split_documents(text_document)
print(texts[0].page_content)
print("-----")
print(texts[1].page_content)
print("-----")

Created a chunk of size 167, which is longer than the specified 100


Langchain is specially used for LLM powered applications.
-----
Langchain  -> Chaining  -> Focus on Sequential Execution of process
* In Langchain, everything happens in sequential order So we call it as Directed Acyclic Graph(DAG)
-----


In [24]:
# Local Weaviate Vector store
from langchain_weaviate.vectorstores import WeaviateVectorStore

local_weaviate_db = WeaviateVectorStore.from_documents(texts, embeddings_google, client=local_client)
local_weaviate_db

In [12]:
query = "What is langchain?"
docs = local_weaviate_db.similarity_search(query)

In [25]:
# Cloud Weaviate Vector Store
from weaviate.classes.init import Auth
import os

# Best practice: store your credentials in environment variables
weaviate_url = os.environ["WEAVIATE_URL"]
weaviate_api_key = os.environ["WEAVIATE_API_KEY"]

weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
    skip_init_checks=True
)

print(weaviate_client.is_ready()) 

True


In [26]:
cloud_db = WeaviateVectorStore.from_documents(texts, embeddings_google, client=weaviate_client)

In [27]:
query = "What is langchain?"
docs = cloud_db.similarity_search(query)
docs

[Document(metadata={'source': '../data/sample.txt'}, page_content='Langchain  -> Chaining  -> Focus on Sequential Execution of process\n* In Langchain, everything happens in sequential order So we call it as Directed Acyclic Graph(DAG)'),
 Document(metadata={'source': '../data/sample.txt'}, page_content='Langchain is specially used for LLM powered applications.'),
 Document(metadata={'source': '../data/sample.txt'}, page_content='LangGraph: We can create stateful Multi AI Agents Applications where Agents will be communicating with themselves to solve complex workflow.\n* They follow the graph structure and it is also going to maintain information which will be shared within the Nodes that is why it is called stategraph.')]

In [14]:
# Local Weaviate as Retriever
# Maximal marginal relevance search (mmr)
retriever = local_weaviate_db.as_retriever(search_type="mmr")
retriever.invoke(query)[0]

Document(metadata={'source': '../data/sample.txt'}, page_content='Langchain is specially used for LLM powered applications.')

In [29]:
# Cloud Weaviate as Retriever
# Maximal marginal relevance search (mmr)
cloud_retriever = cloud_db.as_retriever(search_type="mmr")
cloud_retriever.invoke(query)[0]

Document(metadata={'source': '../data/sample.txt'}, page_content='Langchain is specially used for LLM powered applications.')

In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [16]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [22]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [23]:
rag_chain.invoke("What is LangGraph?")

"LangGraph is a system for creating stateful multi-AI agent applications.  These agents communicate to solve complex workflows, following a graph structure and sharing information between nodes.  This contrasts with Langchain's sequential, DAG-based approach."

In [30]:
# Coud Weaviate RAG
cloud_rag_chain = (
    {"context": cloud_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [31]:
cloud_rag_chain.invoke("What is LangGraph?")

"LangGraph is a system for creating stateful, multi-AI agent applications.  These agents communicate to solve complex workflows, following a graph structure and sharing information between nodes.  This contrasts with Langchain's sequential, DAG-based approach."